## Punto 1

In [12]:
from html.parser import HTMLParser

## Punto 2

In [1]:
import duckdb
import pandas as pd
import sqlite3

In [4]:
## Cargo tablas en memoria

df = pd.read_excel("bd.xlsx", sheet_name=None)

clientes = df["CLIENTES"]
transac = df["TRANSACCIONES"]
cat_consumo = df["CATEGORIAS_CONSUMO"]

transac.dtypes

IDENTIFICACION        object
ID_TRANSACCION         int64
FECHA_TRANSACCION     object
CODIGO_CATEGORIA       int64
ESTADO                object
VALOR_COMPRA         float64
dtype: object

In [5]:
## Casteo fecha

transac["FECHA_TRANSACCION"] = pd.to_datetime(transac["FECHA_TRANSACCION"], yearfirst=True )
transac.dtypes

IDENTIFICACION               object
ID_TRANSACCION                int64
FECHA_TRANSACCION    datetime64[ns]
CODIGO_CATEGORIA              int64
ESTADO                       object
VALOR_COMPRA                float64
dtype: object

In [6]:
## Envio tablas a la db

conn = sqlite3.connect("db_tuya.db")

clientes.to_sql("clientes", conn, index=False)
transac.to_sql("transacciones", conn, index=False)
cat_consumo.to_sql("categorias_consumo", conn, index=False)

60

In [ ]:
# Cito textual del enunciado: "Categoría de consumo preferida dentro de cada cadena"

## VOY A SUPONER QUE CON "CADENA" SE REFIEREN A CIUDAD PUESTO QUE NO HAY OTRA COSA CON QUE ASOCIAR
# UNA MISMA CATEGORIA EN LA TABLA "CATEGORIAS_CONSUMO"

In [ ]:
## Todas las consultas juntas

duckdb.sql("""
WITH ventana_fecha_max AS (
    SELECT IDENTIFICACION, CODIGO_CATEGORIA, FECHA_TRANSACCION, MAX(FECHA_TRANSACCION) OVER(
        PARTITION BY IDENTIFICACION, CODIGO_CATEGORIA
    ) AS Ultima_transac
    FROM transac
),

prefer_cliente AS (
    SELECT IDENTIFICACION, CODIGO_CATEGORIA, Ultima_transac, COUNT(*) AS Cantidad_transac
    FROM ventana_fecha_max
    GROUP BY IDENTIFICACION, CODIGO_CATEGORIA, Ultima_transac
),

cols_necesarias AS (
    SELECT IDENTIFICACION, NOMBRE_CATEGORIA, CIUDAD, Cantidad_transac, Ultima_transac
    FROM prefer_cliente p
    JOIN cat_consumo c
    ON p.CODIGO_CATEGORIA = c.CODIGO_CATEGORIA
),

rankeo AS (
    SELECT *, DENSE_RANK() OVER (
        PARTITION BY CIUDAD
        ORDER BY Cantidad_transac DESC
        ) AS Rank
    FROM cols_necesarias
)


SELECT IDENTIFICACION, NOMBRE_CATEGORIA, CIUDAD, Ultima_transac
FROM rankeo
WHERE Rank = 1

""")

# Solo se debe cambiar en el where el "Rank"
# Rank=1 para primeras preferidas, Rank=2 para segundas preferidas...

┌───────────────────┬──────────────────┬──────────┬─────────────────────┐
│  IDENTIFICACION   │ NOMBRE_CATEGORIA │  CIUDAD  │   Ultima_transac    │
│      varchar      │     varchar      │ varchar  │    timestamp_ns     │
├───────────────────┼──────────────────┼──────────┼─────────────────────┤
│ 2I3J4K5L6M7N8O9P0 │ Ropa             │ Medellín │ 2020-09-10 00:00:00 │
│ 2G3H4I5J6K7L8M9N0 │ Belleza          │ Medellín │ 2020-11-05 00:00:00 │
│ 2Q3R4S5T6U7V8W9X0 │ Mascotas         │ Medellín │ 2015-09-15 00:00:00 │
│ 2Y3Z4A5B6C7D8E9F0 │ Mascotas         │ Bogotá   │ 2009-11-25 00:00:00 │
│ 2G3H4I5J6K7L8M9N0 │ Automotriz       │ Bogotá   │ 2019-09-25 00:00:00 │
│ 2K3L4M5N6O7P8Q9R0 │ Belleza          │ Cali     │ 2023-03-05 00:00:00 │
│ 2A3B4C5D6E7F8G9H0 │ Libros           │ Cali     │ 2014-09-10 00:00:00 │
│ 2Y3Z4A5B6C7D8E9F0 │ Música           │ Cali     │ 2006-11-10 00:00:00 │
└───────────────────┴──────────────────┴──────────┴─────────────────────┘

## Punto 3

In [ ]:
## Cargo tablas en memoria

dic = pd.read_excel("rachas.xlsx", sheet_name=None)

retiros = dic["retiros"]
historia = dic["historia"]

In [7]:
retiros.dtypes

identificacion            object
fecha_retiro      datetime64[ns]
dtype: object

In [8]:
historia.dtypes

identificacion            object
corte_mes         datetime64[ns]
saldo                      int64
dtype: object

In [ ]:
## Envio las tablas a la db

retiros.to_sql("retiros", conn, index=False)
historia.to_sql("historia", conn, index=False)

2925

In [ ]:
## Todas las consultas juntas

duckdb.sql("""
WITH jn AS (
    SELECT r.identificacion, corte_mes, saldo, fecha_retiro
    FROM historia r
    LEFT JOIN retiros h
    ON r.identificacion = h.identificacion
),
    
niveles AS (
    SELECT identificacion, saldo, corte_mes, fecha_retiro, COUNT(*) OVER (
        PARTITION BY identificacion, corte_mes
    ) AS conteo,
        CASE
            WHEN conteo = 1 AND fecha_retiro IS NULL THEN 'N0'
            WHEN conteo = 1 AND corte_mes <= fecha_retiro  THEN 'N0'
            WHEN saldo < 300000  THEN 'N0'
            WHEN saldo < 1000000  THEN 'N1'
            WHEN saldo < 3000000  THEN 'N2'
            WHEN saldo < 5000000  THEN 'N3'
            ELSE 'N4'
        END AS nivel
    FROM jn
),

meses_consec AS (
    SELECT identificacion, corte_mes, nivel, DATEDIFF('month', '1970-01-01', corte_mes)+ROW_NUMBER() OVER(
        PARTITION BY identificacion, nivel
        ORDER BY corte_mes DESC
    ) AS consecutivo
    FROM niveles
),

ventanas AS (
    SELECT identificacion, nivel, corte_mes, COUNT(*)  OVER(
        PARTITION BY identificacion, consecutivo, nivel
    ) AS racha,
    MAX(corte_mes) OVER(
        PARTITION BY identificacion, consecutivo, nivel
    ) AS fecha_fin
    FROM meses_consec
)



SELECT identificacion, MAX(racha) AS racha,
    (
        SELECT fecha_fin
        FROM ventanas sub
        WHERE sub.identificacion = v.identificacion
            AND sub.racha = MAX(v.racha)
        ORDER BY fecha_fin DESC  -- SI LA RACHA MAS LARGA ESTA DUPLICADA... SE TOMA LA FECHA MAS RECIENTE
        LIMIT 1
    ) AS fecha_fin,
    nivel
    
FROM ventanas v
GROUP BY identificacion, nivel

""")

┌───────────────────┬───────┬─────────────────────┬─────────┐
│  identificacion   │ racha │      fecha_fin      │  nivel  │
│      varchar      │ int64 │    timestamp_ns     │ varchar │
├───────────────────┼───────┼─────────────────────┼─────────┤
│ 09SYGN7IXDQV5X9IP │     7 │ 2023-07-31 00:00:00 │ N0      │
│ 0TQ17VI65FDDKT227 │     6 │ 2024-04-30 00:00:00 │ N0      │
│ 3QWPW0GFU2JIA5UE4 │     8 │ 2023-08-31 00:00:00 │ N0      │
│ 9NWFWJ38XN31WBFXG │     1 │ 2024-12-31 00:00:00 │ N3      │
│ 9NWFWJ38XN31WBFXG │     5 │ 2024-01-31 00:00:00 │ N0      │
│ A4JYPUIFCGXQSH8XL │    19 │ 2024-07-31 00:00:00 │ N0      │
│ AXH5M6W00RW6A1MRV │     7 │ 2024-08-31 00:00:00 │ N0      │
│ DLRDFJ40BGVP9H10W │     5 │ 2023-07-31 00:00:00 │ N0      │
│ F5AUCVPQMIXFMLKNY │    11 │ 2024-12-31 00:00:00 │ N0      │
│ FA2OC6FGIMI4S37PU │    11 │ 2024-06-30 00:00:00 │ N0      │
│         ·         │     · │          ·          │ ·       │
│         ·         │     · │          ·          │ ·       │
│       